# Prioridades:
+ Parâmetros utilizados no classificador.

+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

+ Visualização: Separar o conjunto de test em 2 ou 3 arquivos e visualizar o que o modelo classificou e o que os anotadores classificaram (separar por id do anotador).

+ Incluir POS tagging.

+ Tranformar o dado $x_i$ em um $x'_i$ que incorpora os 2 últimos e próximos tokens.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from collections import Counter

os.chdir("mock")

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*/**/***/****.{}'.format(extension))]

In [3]:
all_filenames[0]

'161704902/[PRATICA_ETAPA_1]/Documentos/20180510_Rcl_22328_314302526.ner.csv'

# Cria uma tag de inicio e fim de arquivo em cada arquivo antes de apendar todos os arquivos

In [4]:
frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN')
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ']
    frames.append(df)
    
combined_csv = pd.concat(frames)
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [5]:
combined_csv.head() , combined_csv.tail()
print("Número de linhas dos arquivos concatenados:", len(combined_csv['Tag']))

Número de linhas dos arquivos concatenados: 967910


In [6]:
combined_csv.reset_index(inplace=True, drop=True)

In [7]:
combined_csv[-10:]

,Token,Tag
967900,Chefe,O
967901,,O
967902,do,O
967903,,O
967904,Plenário,O
967905,\n,O
967906,id,O
967907,:,O
967908,,O
967909,20141203_ADI_4350_285683668,FIM_ARQ


# Encontra parágrafo duplo no arquivo. Uma opção de separar por sentenças.

In [9]:
a_df = combined_csv #Simplifica o nome do arquivo 
starts = a_df[a_df['Token']=='\n'].index & a_df[a_df['Token'].shift(-1)=='\n'].index #Identifica os paragrafos duplos
print(u'Padrões(sentenças) encontrados:', len(starts))

Padrões(sentenças) encontrados: 9708


In [10]:
combined_csv.iloc[:starts[0]+2] # Primeira sentença
combined_csv.iloc[starts[-1]+2:] # Última sentença

,Token,Tag
967898,Assessora,O
967899,-,O
967900,Chefe,O
967901,,O
967902,do,O
967903,,O
967904,Plenário,O
967905,\n,O
967906,id,O
967907,:,O


In [11]:
i = 1
combined_csv.iloc[starts[i-1]+2:starts[i]+2]

,Token,Tag
14,RECLAMAÇÃO,O
15,,O
16,22.328,O
17,,O
18,RIO,O
19,,O
20,DE,O
21,,O
22,JANEIRO,O
23,,O


In [12]:
import time

start = time.time()
combined_csv['Sentence #'] = 'Sentence'

combined_csv['Sentence #'][:starts[0]+2] = 'Sentence %d'%(1) # Primeira sentença
combined_csv['Sentence #'][starts[-1]+2:] = 'Sentence %d'%(len(starts)+1) # Última sentença

for i in range(1,len(starts)):
    combined_csv['Sentence #'][starts[i-1]+2:starts[i]+2] = 'Sentence %d'%(i+1) 
combined_csv.head(), combined_csv.tail()
end = time.time()

elapsed_time = time.time() - start
'tempo',time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

('tempo', '00:05:35')

In [13]:
len(combined_csv['Sentence #'].unique())

9709

In [14]:
combined_csv.head()

,Token,Tag,Sentence #
0,Ementa,INICIO_ARQ,Sentence 1
1,,O,Sentence 1
2,e,O,Sentence 1
3,,O,Sentence 1
4,Acórdão,O,Sentence 1


# Modelo

In [15]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [16]:
X = combined_csv.drop('Tag', axis=1) # Define o conjunto X
v = DictVectorizer(sparse=True) # Função que transforma listas de features em vetores
X = v.fit_transform(X.to_dict('records')) #Aplica a função de vetorização no conjunto 
                                          #X que foi colocado no formato 'records' (informa o que preenche cada coluna 
                                          # da linha i)
y = combined_csv.Tag.values # Define o conjunto y

classes = np.unique(y) # Define quais serão as classes baseado nos valores únicos da coluna y
classes = classes.tolist() # Tranforma as classes de array para lista

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0) # Divide o conjunto em treino
                                                                                            #e teste
X_train.shape, y_train.shape # Formato dos dados

((648499, 36129), (648499,))

In [17]:
# Removendo a tag 'O'
new_classes = classes.copy()
new_classes.pop()
new_classes

['B_Doutrina',
 'B_Doutrinador',
 'B_Pessoa',
 'B_Precedente',
 'B_Ref. Legislativa',
 'FIM_ARQ',
 'INICIO_ARQ',
 'I_Doutrina',
 'I_Doutrinador',
 'I_Pessoa',
 'I_Precedente',
 'I_Ref. Legislativa']

In [18]:
# Separa as frases para criar contexto na aprendizagem

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['Token'].values.tolist(),
                                                     s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

In [19]:
def word2features(sent, i):
    word = sent[i][0]
#     postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
#         'postag': postag,
#         'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
#         postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
#             '-1:postag': postag1,
#             '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
#         print(i,type(word1),word1)
#         postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
#             '+1:postag': postag1,
#             '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [20]:
combined_csv['Token'] = combined_csv['Token'].astype('str')
getter = SentenceGetter(combined_csv)
sentences = getter.sentences

In [21]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [22]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Gradient Descent
    c1=0.1, # coefficient for L1 penalty
    c2=0.1, # coefficient for L2 penalty
    max_iterations=100,
    all_possible_transitions=True) # whether to include transitions that are possible, but not observed

crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=new_classes)

0.8184949279966993

In [23]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

                    precision    recall  f1-score   support

        B_Doutrina       0.77      0.62      0.69        99
     B_Doutrinador       0.70      0.45      0.55        87
          B_Pessoa       0.70      0.64      0.67       807
      B_Precedente       0.85      0.75      0.80       913
B_Ref. Legislativa       0.82      0.72      0.76       892
           FIM_ARQ       1.00      0.71      0.83        59
        INICIO_ARQ       1.00      0.72      0.84        58
        I_Doutrina       0.76      0.81      0.78      4225
     I_Doutrinador       0.77      0.65      0.70      3565
          I_Pessoa       0.84      0.76      0.80      4701
      I_Precedente       0.88      0.84      0.86     13490
I_Ref. Legislativa       0.87      0.83      0.85     12040

         micro avg       0.84      0.80      0.82     40936
         macro avg       0.83      0.71      0.76     40936
      weighted avg       0.84      0.80      0.82     40936

